In [57]:
%cd AdventOfCode2023

[WinError 2] The system cannot find the file specified: 'AdventOfCode2023'
D:\Jupyter\AdventOfCode2023


# Advent Of Code 2023
## Day 1: Trebuchet?!

In [23]:
# Part 1
def extract_number(from_line: str) -> int:
    clean_line = ''.join(char for char in from_line if char.isdigit())
    return int(clean_line[0] + clean_line[-1])

with open('files/day1-input', 'r', encoding='utf-8') as input_file:
    print(f"The sum of all the calibration values is {sum(extract_number(line) for line in input_file)}")

The sum of all the calibration values is 55447


In [58]:
# Part 2
DIGIT_REPLACEMENT = {
    'one': 1,
    'two': 2,
    'three': 3,
    'four': 4,
    'five': 5,
    'six': 6,
    'seven': 7,
    'eight': 8,
    'nine': 9
}

def extract_number(from_line: str) -> int:
    
    first_digit, last_digit = None, None

    for (index, char) in enumerate(from_line):
        
        # Match regular digit
        if char.isdigit():
            last_digit = char
        
        else:
            # Match spelled out digit
            matches = [value for digit, value in DIGIT_REPLACEMENT.items() if from_line[index:].startswith(digit)]
            if any(matches):
                last_digit = str(matches[0])
    
        if first_digit is None:
            first_digit = last_digit
            
    return int(first_digit + last_digit)

with open('files/day1-input', 'r', encoding='utf-8') as input_file:
    print(f"The sum of all the calibration values is {sum(extract_number(line) for line in input_file)}")

The sum of all the calibration values is 54706
